In [ ]:
import pandas as pd
import numpy as np
import fix_yahoo_finance as fyf
from pandas_datareader import data as web
from datetime import datetime

In [ ]:
#fetching data
fyf.pdr_override()
today_date=datetime.today().strftime('%Y-%m-%d')
yf_data=web.get_data_yahoo('BTC-USD', start = '2017-01-01', end = today_date)
#bitcoin dataframe
bitcoin_df=pd.DataFrame(yf_data)
bitcoin_df.reset_index(inplace=True)
bitcoin_df

[*********************100%***********************]  1 of 1 downloaded


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...,...
1500,2021-02-14,47114.507813,49487.640625,47114.507813,48717.289063,48717.289063,71248675228
1501,2021-02-15,48696.535156,48875.570313,46347.476563,47945.058594,47945.058594,77069903166
1502,2021-02-17,49207.277344,52533.914063,49072.378906,52149.007813,52149.007813,80820545404
1503,2021-02-18,52140.972656,52474.105469,51015.765625,51679.796875,51679.796875,52054723579


In [ ]:
#checking for null values
bitcoin_df.isnull().values.any()

False

**Feature Engineering**

In [ ]:
#simple moving average 10,30 days
def SMA(df, x):
  SMA_calc=df.Close.rolling(window=x, min_periods=x).mean()
  return SMA_calc
bitcoin_df['SMA10']=SMA(bitcoin_df,10)
bitcoin_df['SMA30']=SMA(bitcoin_df,30)

#exponential moving average 20,30,200 days
def EMA(df, x):
  EMA_calc=df.Close.ewm(span=x, min_periods=x, adjust=False).mean()
  return EMA_calc

bitcoin_df['EMA20']=EMA(bitcoin_df,20)
bitcoin_df['EMA30']=EMA(bitcoin_df,30)
bitcoin_df['EMA200']=EMA(bitcoin_df,200)
#bitcoin_df['EMA20']=bitcoin_df.Close.ewm(span=20, min_periods=20, adjust=False).mean()

In [ ]:
#rate of change ROC 10,30 days
def ROC(df, n):
  ROC_diff=df.Close.diff(n)
  ROC_shift=df.Close.shift(n)
  #roc formula
  ROC_calc=((ROC_diff/ROC_shift)*100)
  return ROC_calc
  
bitcoin_df['ROC10']=ROC(bitcoin_df,10)
bitcoin_df['ROC30']=ROC(bitcoin_df,30)

In [ ]:
#volume rate of change 10,30 days
def VROC(df, n):
  VROC_diff=df.Volume.diff(n)
  VROC_shift=df.Volume.shift(n)
  #roc formula
  VROC_calc=((VROC_diff/VROC_shift)*100)
  return VROC_calc
  
bitcoin_df['VROC10']=VROC(bitcoin_df,10)
bitcoin_df['VROC30']=VROC(bitcoin_df,30)

In [ ]:
#relative strength index RSI 14 days
def RSI(df, n=14):
  delta=df.Close.diff()
  dup, ddown=delta.copy(), delta.copy()
  #average gains and losses
  dup[dup<0]=0
  ddown[ddown>0]=0
  rolup=dup.rolling(window=n, min_periods=n).mean()
  roldown=ddown.rolling(window=n, min_periods=n).mean().abs()
  #relative strength
  RS_calc=rolup/roldown
  #rsi formula
  RSI_calc=(100-(100/(1+RS_calc)))
  return RSI_calc

bitcoin_df['RSI14']=RSI(bitcoin_df)

In [ ]:
#RSI indicators %30 %70
def RSI_Ind(df):
  rsi_vals=[]
  for i in df['RSI14'].iloc[:]:
    if i <= 30:
      rsi_vals.append(0)
    elif i >= 70:
      rsi_vals.append(1)
    else:
      rsi_vals.append(2)
  return rsi_vals

bitcoin_df['RSI14_IND']=RSI_Ind(bitcoin_df)

In [ ]:
#STO%K%D Stochastic oscillator 14,200 days
def STOK(close, low, high, n):
  STOK_calc=((close-low.rolling(n).min())/(high.rolling(n).max()-low.rolling(n).min()))*100
  return STOK_calc

bitcoin_df['STOK14']=STOK(bitcoin_df.Close, bitcoin_df.Low, bitcoin_df.High, 14)
bitcoin_df['STOK200']=STOK(bitcoin_df.Close, bitcoin_df.Low, bitcoin_df.High, 200)

def STOD(stod, n):
  STOD_calc=stod.rolling(3).mean()
  return STOD_calc

bitcoin_df['STOD14']=STOD(bitcoin_df.STOK14, 14)
bitcoin_df['STOD14']=STOD(bitcoin_df.STOK200, 200)

In [ ]:
#bollinger bands upper
def BBandsUp(close, n, mult):
  BBandsUp_calc=close.rolling(window=n, min_periods=n).mean() + close.rolling(window=n, min_periods=n).std() * mult
  return BBandsUp_calc

bitcoin_df['BBandUpper']=BBandsUp(bitcoin_df.Close,20,2)
#bollinger bands lower
def BBandsLow(close, n, mult):
  BBandsLow_calc=close.rolling(window=n, min_periods=n).mean() - close.rolling(window=n, min_periods=n).std() * mult
  return BBandsLow_calc

bitcoin_df['BBandlower']=BBandsLow(bitcoin_df.Close,20,2)

In [ ]:
bitcoin_df.head(32)

,Date,Open,High,Low,Close,Adj Close,Volume,SMA10,SMA30,EMA20,EMA30,EMA200,ROC10,ROC30,VROC10,VROC30,RSI14,RSI14_IND,STOK14,STOK200,STOD14,BBandUpper,BBandlower
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
5,2017-01-06,1014.239990,1046.810059,883.943970,902.200989,902.200989,351876000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
6,2017-01-07,903.487000,908.585022,823.556030,908.585022,908.585022,279550016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
7,2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
8,2017-01-09,913.244019,913.685974,879.807007,902.828003,902.828003,141876992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
9,2017-01-10,902.440002,914.872986,901.059998,907.679016,907.679016,115808000,976.451697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
